https://hostadvice.com/blog/ai/types-of-ollama-models/

In [1]:
from langchain_ollama import OllamaLLM
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report

Pré-processamento

In [2]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

print(f"Test set: {len(test)} exemplos, classes: {test['classe'].value_counts().to_dict()}")
#train_oversampling = pd.read_csv('train_oversampling.csv') acho que nao é preciso 

Test set: 1858 exemplos, classes: {0: 1273, -1: 417, 1: 168}


LLM

In [3]:
'''escolher entre:
Meta Llama 3.1 8B-Instruct
Qwen 3-8
JetMoE-8B
FuxiTranyu-8B
GPT-J-6B'''

'escolher entre:\nMeta Llama 3.1 8B-Instruct\nQwen 3-8\nJetMoE-8B\nFuxiTranyu-8B\nGPT-J-6B'

In [ ]:
llm = OllamaLLM(model="llama3.1:8b", temperature=0.1)

def extract_label(response):

    print(response)
    resp = str(response).strip().lower()
    
    #se o modelo classificar como deve
    for num in ['-1', '0', '1']:
        if num in resp:
            return int(num)
    
    #fallback caso o modelo se passe 
    if any(word in resp for word in ['citação', 'citacao']):
        return -1
    if any(word in resp for word in ['facto', 'fato']):
        return 0
    if any(word in resp for word in ['viés', 'vies', 'tendencioso']):
        return 1
    
    return 0


def generate_predictions(test_df, prompt_template, max_examples=5):
    predictions = []

    examples = train.sample(max_examples).to_dict('records') if max_examples > 0 else []

    for idx, row in test_df.iterrows():
        text = row['sentences']
        true_label = row['classe']

        prompt = prompt_template.format(
            text=text,
            examples=examples,
            label_map="-1: Citação | 0: Facto | 1: Viés"
        )

        response = llm.invoke(prompt)
        pred_label = extract_label(response)

        predictions.append({
            'id': idx,
            'text': text,
            'true_label': true_label,
            'predicted_label': pred_label,
            'raw_response': str(response),
            'prompt_used': prompt_template[:100] + "..."
        })

    return pd.DataFrame(predictions)

def get_examples():
    
    return

Zero-Shot

In [15]:
zero_shot_prompt = """
Tu és um classificador de texto. Classifica o seguinte texto em português brasileiro:
{label_map}

Texto: {text}

Responde APENAS com -1, 0 ou 1 (sem mais texto):
"""

zero_shot_results = generate_predictions(test, zero_shot_prompt, max_examples=0)
print(zero_shot_results[['true_label','predicted_label']].head())

0
0
-1
0
0
0
0
-1
0
-1
0: Facto
-1
-1
0
-1
0: Facto
0
-1
0
0
0
0
0
-1
0
0
-1
0
0
-1
0: Facto
-1
0
-1
0
0: Facto
-1
-1
0
0
0
0
0
0
-1
0
0
-1
0
0
0
0
0
-1
0
-1
0
-1
0
0
-1
0
-1
0
0
0
0
-1
0
-1
0
0
0
0
-1
-1
-1
-1
0
0
0
0
0
0
0
0
-1
0
0
0
0
0: Facto
0
-1
0
0
0
0
0: Facto
0
0: Facto
0
-1
-1
0
0
0: Facto
Classificação: 0 (Facto)
0
-1
0
0
-1
0
0
0: Facto
0
0
0
-1
0
0
0
0: Facto
0
0
0
-1. A resposta parece ser uma reação pessoal e emocional à entrada do usuário, em vez de um fato objetivo ou uma declaração imparcial.
-1: Citação
0: Facto
0
-1
-1
-1
-1
-1
0
0
0
0
0
0
-1
0: Facto
0: Facto
Classificação do texto: 1 (Viés)

Justificativa: O texto parece ser uma instrução para os leitores participarem de uma pesquisa, mas a presença da palavra "APENAS" e a ênfase na resposta numérica sugerem que o objetivo é influenciar a percepção dos leitores sobre um determinado assunto.
0
-1
0
0
0
0
0
0
0
0
-1
0
0
0
0
-1
0
0
-1
0: Facto
-1
0
0
0
0
0
0: Facto
0
0
0
-1
0
0
0
-1: Citação
-1
0: Facto
-1
-1
0
0
0
0

In [16]:
print(classification_report(zero_shot_results['true_label'],
                            zero_shot_results['predicted_label'],
                            digits=3))
print("Accuracy:", accuracy_score(zero_shot_results['true_label'],
                                 zero_shot_results['predicted_label']))
print("F1 (weighted):", f1_score(zero_shot_results['true_label'],
                                 zero_shot_results['predicted_label'],
                                 average='weighted'))
pred_1 = zero_shot_results[zero_shot_results['predicted_label'] == 1]
print(len(pred_1))
print(pred_1["text"])

              precision    recall  f1-score   support

          -1      0.459     0.496     0.477       417
           0      0.775     0.855     0.813      1273
           1      0.000     0.000     0.000       168

    accuracy                          0.697      1858
   macro avg      0.411     0.450     0.430      1858
weighted avg      0.634     0.697     0.664      1858

Accuracy: 0.6969860064585576
F1 (weighted): 0.66417475650984
4
145             Pesquisa Ipec com eleitores de São Paulo:
459              Constituinte Eymael (Democracia Cristã):
517     Não se tem por que buscar métodos de criminali...
1430    "É completamente abominável a mais nova mentir...
Name: text, dtype: object


Few-shot

In [19]:
few_shot_prompt = """
Tu és um classificador de texto. Classifica o seguinte texto em português brasileiro:
{label_map}

Tens aqui exemplos para auxiliar a classificação:
{examples}

Texto: {text}

Responde APENAS com -1, 0 ou 1 (sem mais texto):
"""

few_shot_results = generate_predictions(test, few_shot_prompt, max_examples=5)
print(few_shot_results[['true_label','predicted_label']].head())

0
0
-1
0
0
0
0
-1
-1
-1
-1
0
-1
0
0
-1
0
0
0
-1
-1
0
0
0
0
0
0
0
0
-1
0
-1
-1
0
0
-1
0
0
0
-1
0
-1
-1
0
-1
0
0
-1
0
0
0
0
0
0
0
-1
0
0
0
0
-1
0
0
0
0
-1
-1
-1
0
-1
0
0
0
0
0
-1
-1
-1
0
0
0
0
0
0
0
0
0
0
0
-1
0
-1
0
-1
0
0
0
0
-1. A resposta é uma citação de alguém em favor da abolição do assédio eleitoral.
0
0
-1
0
-1
0
0
-1
-1. O texto parece ser uma resposta a um questionário ou formulário, e não contém opinião ou informação factual. Portanto, é uma citação.
0
-1
0
0
-1
0
0
0
0
0
0
-1
0
0
0
-1. A resposta parece ser uma opinião pessoal e não um fato objetivo, mas também não é uma citação explícita. Portanto, classifico como "Citação" (-1).
0
0
0
-1
-1
-1
0
-1
0
-1
-1
-1
0
0
0
0
-1
-1
-1
-1. A citação é uma frase de um livro ou poema, e não contém opinião ou informação factual.
0
-1. O texto parece ser uma pergunta direta ao leitor para classificar a pesquisa Ipec, sem conteúdo específico que possa ser considerado como fato, viés ou citação.
0
0
0
0
0
0
0
0
0
-1
-1
0
0
0
0
-1
0
0
0
-1

In [20]:
print(classification_report(few_shot_results['true_label'],
                            few_shot_results['predicted_label'],
                            digits=3))
print("Accuracy:", accuracy_score(few_shot_results['true_label'],
                                 few_shot_results['predicted_label']))
print("F1 (weighted):", f1_score(few_shot_results['true_label'],
                                 few_shot_results['predicted_label'],
                                 average='weighted'))
pred_1 = few_shot_results[few_shot_results['predicted_label'] == 1]
print(len(pred_1))
print(pred_1["text"])

              precision    recall  f1-score   support

          -1      0.545     0.659     0.597       417
           0      0.801     0.848     0.824      1273
           1      0.000     0.000     0.000       168

    accuracy                          0.729      1858
   macro avg      0.449     0.503     0.474      1858
weighted avg      0.671     0.729     0.699      1858

Accuracy: 0.7292787944025835
F1 (weighted): 0.6985190939030929
5
723     Opositores investiram na tag "Bolsonaro corrup...
757     Para o partido de Lula, a Jovem Pan é parcial ...
1017    Agora fizeram uma que extrapolou todos os limi...
1419    O PT e o Lula representam tudo o que o nosso p...
1800    Além disso, adepto de corrente ideológica de e...
Name: text, dtype: object


Chain of thought

In [ ]:
chain_of_thought_prompt = """
Tu és um classificador de texto. Classifica o seguinte texto em português brasileiro:
{label_map}

Tens aqui exemplos para auxiliar a classificação:
{examples}

Texto: {text}

Responde APENAS com -1, 0 ou 1 (sem mais texto):
"""

chain_of_thought_results = generate_predictions(test, chain_of_thought_prompt, max_examples=5)
print(chain_of_thought_results[['true_label','predicted_label']].head())

In [ ]:
print(classification_report(chain_of_thought_results['true_label'],
                            chain_of_thought_results['predicted_label'],
                            digits=3))
print("Accuracy:", accuracy_score(chain_of_thought_results['true_label'],
                                 chain_of_thought_results['predicted_label']))
print("F1 (weighted):", f1_score(chain_of_thought_results['true_label'],
                                 chain_of_thought_results['predicted_label'],
                                 average='weighted'))
chain_of_thought_pred_1 = chain_of_thought_results[chain_of_thought_results['predicted_label'] == 1]
print(len(chain_of_thought_pred_1))
print(chain_of_thought_pred_1["text"])

Zero shot chain of thought

Self consistency(?)

Model 2